In [3]:
import pandas as pd
df_amazon=pd.read_csv('../MLOpsReviews/amazon_prime_titles.csv')
df_disney=pd.read_csv('../MLOpsReviews/disney_plus_titles.csv')
df_hulu=pd.read_csv('../MLOpsReviews/hulu_titles.csv')
df_netflix=pd.read_csv('../MLOpsReviews/netflix_titles.csv')

In [4]:
df1 = pd.read_csv('../MLOpsReviews/ratings/1.csv')
df2 = pd.read_csv('../MLOpsReviews/ratings/2.csv')
df3 = pd.read_csv('../MLOpsReviews/ratings/3.csv')
df4 = pd.read_csv('../MLOpsReviews/ratings/4.csv')
df5 = pd.read_csv('../MLOpsReviews/ratings/5.csv')
df6 = pd.read_csv('../MLOpsReviews/ratings/6.csv')
df7 = pd.read_csv('../MLOpsReviews/ratings/7.csv')
df8 = pd.read_csv('../MLOpsReviews/ratings/8.csv')

##### Transformaciones: Para este MVP no necesitas perfección, ¡necesitas rapidez! ⏩ Vas a hacer estas, y solo estas, transformaciones a los datos:

In [5]:
# Generar campo id: Cada id se compondrá de la primera letra del nombre de la plataforma, 
# seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = as123)
df_amazon['show_id'] = 'a'+df_amazon['show_id']
df_disney['show_id'] = 'd'+df_disney['show_id']
df_hulu['show_id'] = 'h'+df_hulu['show_id']
df_netflix['show_id'] = 'n'+df_netflix['show_id']


In [6]:
# Une los dataframes de plataformas
df_plataforma = pd.concat([df_amazon,df_disney,df_hulu,df_netflix],ignore_index=True)

In [7]:
# Los valores nulos del campo rating deberán reemplazarse por el string “G” (corresponde al maturity rating: “general for all audiences”
import numpy as np
df_plataforma['rating'] = df_plataforma['rating'].replace(np.nan,'G')
df_plataforma['duration'] = df_plataforma['duration'].replace(np.nan,'0')

In [8]:
# De haber fechas, deberán tener el formato AAAA-mm-dd
df_plataforma.date_added = pd.to_datetime(df_plataforma.date_added)

In [9]:
# Los campos de texto deberán estar en minúsculas, sin excepciones
df_plataforma = df_plataforma.apply(lambda x: x.str.lower() if x.dtype=='object' else x)

In [10]:
# Actualizo la columna "duration" con los valores que se corrieron a la izquierda ("rating")
mascara_min = df_plataforma[df_plataforma.rating.str.contains('min')].assign(duration=df_plataforma['rating']).assign(rating=pd.np.nan)
mascara_min['rating'] = mascara_min['rating'].replace(np.nan,'G')
df_plataforma.drop(index=mascara_min.index, inplace=True)
df_plataforma = pd.concat([df_plataforma,mascara_min])
mascara_season = df_plataforma[df_plataforma.rating.str.contains('eason')].assign(duration=df_plataforma['rating']).assign(rating=pd.np.nan)
mascara_season['rating'] = mascara_season['rating'].replace(np.nan,'G')
df_plataforma.drop(index=mascara_season.index, inplace=True)
df_plataforma = pd.concat([df_plataforma,mascara_season])

C:\Users\libra\AppData\Local\Temp\ipykernel_13036\4073943326.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  mascara_min = df_plataforma[df_plataforma.rating.str.contains('min')].assign(duration=df_plataforma['rating']).assign(rating=pd.np.nan)
C:\Users\libra\AppData\Local\Temp\ipykernel_13036\4073943326.py:6: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  mascara_season = df_plataforma[df_plataforma.rating.str.contains('eason')].assign(duration=df_plataforma['rating']).assign(rating=pd.np.nan)


In [11]:
# El campo duration debe convertirse en dos campos: duration_int y duration_type. 
# El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)

df_plataforma[['duration_int','duration_type']] = df_plataforma['duration'].str.extract('(\d+)\s(\w+)')
df_plataforma['duration_int'].fillna('0',inplace=True)
df_plataforma['duration_int'] = df_plataforma['duration_int'].astype('int')


In [12]:
# Actualizo los nan de la columna "duration_type" y le pongo sin dato
df_plataforma['duration_type']=df_plataforma['duration_type'].replace(np.nan,'sin dato')

In [13]:
# Normalizo los valores de seasons (plural) para que tengan el valor solo de season (singular)
df_plataforma['duration_type']=df_plataforma['duration_type'].replace('seasons','season')

In [14]:
# Uniendo los dataset de score
df_score = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8],ignore_index=True)

In [15]:
# Renombra la columna rating por score
df_score.rename(columns={'rating':'score'},inplace=True)

In [16]:
# Cambia el formato de la columna timestamp a "YYYY-MM-DD HH:mm:SS"
df_score['timestamp'] = pd.to_datetime(df_score['timestamp'], unit='s')

In [17]:
# Agrego columna con promedio score por movieId
df_score['promedio'] = df_score.groupby('movieId')['score'].transform('mean')

In [18]:
# Genero un dataframe con las columnas "movieId" y "promedio" para luego hacer merge con el dataframe de las plataformas
df_score_promedio = df_score.filter(['movieId','promedio'])
df_score_promedio.drop_duplicates(inplace=True)

In [19]:
# Haciendo el join entre los dataframes df_plataforma y df_score_promedio y guardándolo en el dataframe df
df = pd.merge(df_plataforma,df_score_promedio, left_on='show_id', right_on='movieId', how='inner')

In [21]:
# Guarda el dataframe anterior (merge) para Deployar las 4 consultas en Deta Space
df.to_pickle('../MLOpsReviews/fichero/df_plataforma_score.pickle')

In [22]:
# GUARDO POR DF_PLATAFORMA PARA EDA
df_plataforma = df_plataforma.filter(['show_id','title'])
df_plataforma.to_pickle('../MLOpsReviews/fichero/df_plataforma.pickle')

In [23]:
# GUARDO DF_SCORE PARA EDA
df_score = df_score.filter(['userId','score','movieId','timestamp'])
df_score.to_pickle('../MLOpsReviews/fichero/df_score.pickle')